https://gist.github.com/qpwo/c538c6f73727e254fdc7fab81024f6e1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import count
from collections import namedtuple
import os
import json
import pandas as pd
import random
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.autograd import Variable
# from torch.distributions import Categorical
%matplotlib inline

In [2]:
from mcts import MCTS, KlondikeNode

In [3]:
from tuplestate import init_from_solvitaire
from benchmarking import convert_shootme_to_solvitaire_json


def listdir(path):
    ls = os.listdir(path)
    return [os.path.join(path, f) for f in ls]

solved = listdir('./fixtures/shootme/solved/')
solvedmin = listdir('./fixtures/shootme/solvedmin/')
fixtures = solved + solvedmin

def filename_to_klonstate(fname):
    with open(fname) as f:
        solv = convert_shootme_to_solvitaire_json(f.read())
        state = init_from_solvitaire(solv)
    return state

def random_state():
    fname = random.choice(fixtures)
    state = filename_to_klonstate(fname)
    return KlondikeNode(*state)

In [4]:
random.seed(0)
root_state = random_state()
tree = MCTS()
state = root_state

In [5]:
for i in count(1):
    print(i, end=" ")
    for _ in range(50):
        tree.do_rollout(state)
    state = tree.choose(state)
    if state.is_terminal():
        print("TERMINAL")
        print(state.to_pretty_string())

1 2 3 4 5 6 7 8 

RuntimeError: choose called on childless node K7373

K7373

In [ ]:
import cProfile
cProfile.run('for _ in range(50): tree.do_rollout(state)', 'restats')

import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.sort_stats(SortKey.CUMULATIVE).print_stats()

import os
os.remove('restats')